In [59]:
import spacy
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from tqdm import notebook
from datetime import datetime
import ast
import timeit
import tensorflow as tf
from tensorflow.compat.v1 import Session
from tensorflow.python.saved_model import loader
import os
import csv
import re
import logging
import optparse
import dedupe
from unidecode import unidecode
notebook.tqdm.pandas()

D:\Anaconda\lib\site-packages\tqdm\std.py:651: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
nlp2 = spacy.load('../Spacy models/kfold_3', disable=['parser', 'tagger', 'textcat'])
notary = pd.read_csv('../clean_data.csv')
voc = pd.read_csv('vocop_clustered_dutchrank.csv')
#voc = pd.read_csv('vocop-clustered-new.csv', sep='	')
#rangen = pd.read_excel('../vocop_rangen.xlsx', index_col=0)
#voc['dutch_rank'] = [translate_rank(x, rangen) for x in notebook.tqdm(voc['rank'].tolist())]

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17,27,35,36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## NER functions

In [7]:
def NER(text, nlp):
    ''' Takes in a string and uses the specified NLP model to tag entities within the string.
        Returns all entities tagged as PERSON within the string.
    '''
    # Tag text
    doc = nlp(text)
    holder = []
    
    # Append all entities tagged as PERSON
    for ent in doc.ents:
        if ent.label_ == 'PERSON' and ' ' in ent.text:
            holder.append(ent.text)
    return holder

## NEL functions

In [107]:
def clustering(model_dir, notary, voc):
    settings_file = model_dir


    print('importing data ...')
    data_1 = notary.to_dict(orient='index')
    data_2 = voc.to_dict(orient='index')

    def descriptions():
        for dataset in (data_1, data_2):
            for record in dataset.values():
                yield record['description']

    # ## Training

    if os.path.exists(settings_file):
        print('reading from', settings_file)
        with open(settings_file, 'rb') as sf:
            linker = dedupe.StaticRecordLink(sf)

    else:
        raise ValueError('Could not find model')
    # ## Blocking

    # ## Clustering

    # Find the threshold that will maximize a weighted average of our
    # precision and recall.  When we set the recall weight to 2, we are
    # saying we care twice as much about recall as we do precision.
    #
    # If we had more data, we would not pass in all the blocked data into
    # this function but a representative sample.
    f1 = []
    tests = []
    print('clustering...')
    try:
        linked_records = linker.join(data_1, data_2, 0.1)
    except:
        return None

    print('# duplicate sets', len(linked_records))
    # ## Writing Results

    # Write our original data back out to a CSV with a new column called
    # 'Cluster ID' which indicates which records refer to each other.

    cluster_membership = {}
    for cluster_id, (cluster, score) in enumerate(linked_records):
        for record_id in cluster:
            cluster_membership[record_id] = {'Cluster ID': cluster_id,
                                             'Link Score': score}
    clusters = {}
    for x in cluster_membership:
        if cluster_membership[x]['Cluster ID'] not in clusters:
            clusters[cluster_membership[x]['Cluster ID']] = x
        else:
            clusters[cluster_membership[x]['Cluster ID']] = [clusters[cluster_membership[x]['Cluster ID']], x]
    
    matches = [clusters[x] for x in clusters] 
    return matches


In [108]:
def match_neighbour(start, end, true, prev, distance):
    if true == []:
        return (start, end), prev.i
    if prev.i == len(prev.doc) - 1:
        return (start, end), prev.i
    if fuzz.ratio(true[0].lower(), prev.nbor().text.lower()) >= distance:
        return match_neighbour(start, prev.nbor().idx + len(prev.nbor()), true[1:], prev.nbor(), distance)
    else:
        return (start, end), prev.i

def match_finder(row, match, distance, nlp):

    true = match
    doc = nlp(row.text)

    locs = []
    prev = 0
    for token in doc:
        for x in true:
            if token.i > prev and type(x) == str:
                split = x.split(' ')
                if fuzz.ratio(split[0].lower(), token.text.lower()) >= distance:
                    result, prev = match_neighbour(token.idx, token.idx + len(token), split[1:], token, distance)
                    if result not in locs:
                        if fuzz.ratio(row.text[result[0]:result[1]], x) >= distance:
                            locs.append(result)
                    elif len(split) > 1 and fuzz.ratio(split[1].lower(), token.text.lower()) >= distance:
                        result, prev = match_neighbour(token.nbor(-1).idx, token.idx + len(token), split[1:], token, distance)
                        if result not in locs:
                            if fuzz.ratio(row.text[result[0]:result[1]], x) >= distance:
                                locs.append(result)
    entities = [row.text[x[0]:x[1]] for x in locs]
    return entities

def find_entities(row, x, column, nlp):
    if type(x[column]) != float: 
        match = match_finder(row, [x[column]], 80, nlp) 
    else:
        match = ''
    return match

def find_matches(name, knowledgebase, target_column, distance=90):
    ''' Takes in a string containing the name of a person and returns all possible matches from the
        knowledgebase based on fuzzy string matching.
    '''
    matches = knowledgebase[(knowledgebase[target_column].str.lower().astype(str).apply(fuzz.ratio, args=[name.lower()]) >= distance)]
    return matches

def convert_matches(entity, matches, knowledgebase, nlp):
    data_n = {}
    data_v = {}
    row = entity
    holder = []
    notary_date = datetime.strptime(row.datering, '%Y-%m-%d')
    
    for x in range(matches.shape[0]):
        try:
            date1 = datetime.strptime(matches['date_begin_service_complete'].iloc[x], '%Y-%m-%d')
        except:
            date1 = datetime(year=1, month=1, day =1 )
        try:
            date2 = datetime.strptime(matches['date_end_service_complete'].iloc[x], '%Y-%m-%d')
        except:
            date2 = datetime(year=1, month=1, day =1 )
        if abs((notary_date - date1).days) < 90 or abs((notary_date - date2).days) < 90:
            found_ship_out = find_entities(row, matches.iloc[x], 'shipOutward', nlp)
            found_ship_return = find_entities(row, matches.iloc[x], 'shipReturn', nlp)
            found_loc = find_entities(row, matches.iloc[x], 'placeOfOrigin', nlp)
            found_rank = find_entities(row, matches.iloc[x], 'dutch_rank', nlp)
            holder.append({'rank':found_rank, 'location':found_loc, 'found_ship_return': found_ship_return, 'found_ship_out': found_ship_out})

            cluster = int(matches.iloc[x].VOCOP_id)
            name = str(matches.iloc[x].fullNameOriginal)
            rang = str(matches.iloc[x].dutch_rank)
            loc = str(matches.iloc[x].placeOfOrigin)
            ship_out = str(matches.iloc[x].shipOutward) 
            ship_return = str(matches.iloc[x].shipReturn)
            data_v[cluster] = {'name':name, 'rank':rang, 'location':loc, 'ship_out': ship_out, 'ship_return': ship_return}
    
    name = row['name']
    if holder == []:
        rang = None
        location = None
        ship_out= None
        ship_return = None
    else:
        rang = ' | '.join(set([y.lower() for x in holder for y in x['rank']]))
        if rang == '':
            rang = None
        location = ' | '.join(set([y.lower() for x in holder for y in x['location']]))
        if location == '':
            location = None
        ship_out = ' | '.join(set([y.lower() for x in holder for y in x['found_ship_out']]))
        if ship_out == '':
            ship_return = None
        ship_return = ' | '.join(set([y.lower() for x in holder for y in x['found_ship_return']]))
        if ship_return == '':
            ship_return = None
    data_n[entity.uuid] = {'name':name, 'rank':rang, 'location':location, 'ship_out': ship_out, 'ship_return': ship_return}

    d_n = {'index': [x for x in data_n], 'name': [data_n[x]['name'] for x in data_n], 'rank':[data_n[x]['rank'] for x in data_n],
         'location':[data_n[x]['location'] for x in data_n], 'ship_out': [data_n[x]['ship_out'] for x in data_n], 'ship_return': [data_n[x]['ship_return'] for x in data_n]}

    d_v = {'index': [x for x in data_v], 'name': [data_v[x]['name'] for x in data_v], 'rank':[data_v[x]['rank'] for x in data_v],
         'location':[data_v[x]['location'] for x in data_v], 'ship_out': [data_v[x]['ship_out'] for x in data_v], 'ship_return': [data_v[x]['ship_return'] for x in data_v]}

    dedupe_notary = pd.DataFrame(d_n).set_index('index')
    dedupe_voc = pd.DataFrame(d_v).set_index('index')
    return dedupe_notary, dedupe_voc

def RL(entity, knowledgebase, model_dir, nlp):
    ''' Takes in a string containing an entity and returns either a match within the specified pandas 
        knowledgebase or None if no match is present.
    ''' 
    # Find and narrow down matches
    possible_matches = find_matches(entity['name'], knowledgebase, 'fullNameOriginal')
    converted_entities, converted_matches = convert_matches(entity, possible_matches, knowledgebase, nlp)
    clusters = clustering(model_dir, converted_entities, converted_matches)
    
    return clusters

## Final Function

In [109]:
def Pipeline(row, knowledgebase, NER_model, RL_model):
    holder = []
    entities = NER(row.text, NER_model)
    for x in entities:
        row['name'] = x
        holder.append(RL(row, knowledgebase, RL_model, NER_model))
    return holder


In [110]:
start_time = timeit.default_timer()
x = Pipeline(notary.iloc[85], voc, nlp2, 'rl models/1003no')
elapsed = timeit.default_timer() - start_time
print('Seconds for single query: ' + str(elapsed))

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
INFO:dedupe.api:((TfidfNGramSearchPredicate: (0.2, name), TfidfNGramSearchPredicate: (0.6, ship_out)), (ExistsPredicate: (Exists, ship_out), SimplePredicate: (commonThreeTokens, name)))


importing data ...
reading from rl models/1003no
clustering...
# duplicate sets 1


INFO:dedupe.api:((TfidfNGramSearchPredicate: (0.2, name), TfidfNGramSearchPredicate: (0.6, ship_out)), (ExistsPredicate: (Exists, ship_out), SimplePredicate: (commonThreeTokens, name)))


importing data ...
reading from rl models/1003no
clustering...


INFO:dedupe.api:((TfidfNGramSearchPredicate: (0.2, name), TfidfNGramSearchPredicate: (0.6, ship_out)), (ExistsPredicate: (Exists, ship_out), SimplePredicate: (commonThreeTokens, name)))


importing data ...
reading from rl models/1003no
clustering...
Seconds for single query: 15.787028400000054


In [111]:
x

[[['ee21c8bc-bfea-25ee-c6c1-ddf74644012b', '802002']], None, None]

### NEL

In [ ]:
def NEL(entity, knowledgebase, model_dir):
    ''' Takes in a string containing an entity and returns either a match within the specified pandas 
        knowledgebase or None if no match is present.
    ''' 
    # Find and narrow down matches
    possible_matches = find_matches(entity.naam, knowledgebase, 'fullNameOriginal')
    name_dict = knowledgebase.fullNameNormalized.value_counts()
    converted_matches = convert_matches(entity, possible_matches, name_dict)
    if converted_matches == {}:
        return None
    ranked_matches = ranking(converted_matches, model_dir)
    return max(ranked_matches)

def convert_matches(entity, matches, name_dict):
    
    holder = {}
    notary_date = datetime.strptime(entity.datering, '%Y-%m-%d')
    
    # Try to convert the dates for each entry into datetime format
    for x in range(matches.shape[0]):
        try:
            date1 = datetime.strptime(matches['date_begin_service_complete'].iloc[x], '%Y-%m-%d')
        except:
            date1 = datetime(year=1, month=1, day =1 )
        try:
            date2 = datetime.strptime(matches['date_end_service_complete'].iloc[x], '%Y-%m-%d')
        except:
            date2 = datetime(year=1, month=1, day =1 )
            
        # Keep only matches that are within distance days from notary_date
        if abs((notary_date - date1).days) < 90 or abs((notary_date - date2).days) < 90:
            name_ratio = fuzz.ratio(entity.naam, matches['fullNameOriginal'].iloc[x])
            name_count = name_dict[matches['fullNameNormalized'].iloc[x]]
            vocid=matches['VOCOP_id'].iloc[x]
            day_dif = min([abs((notary_date - date1).days), abs((notary_date - date2).days)])
            location = len(match_finder(entity.text, [matches['placeOfOrigin'].iloc[x]], 80))
            rank = len(match_finder(entity.text, [matches['dutch_rank'].iloc[x]], 80))
            numships = len(match_finder(entity.text, [matches['shipOutward'].iloc[x], matches['shipReturn'].iloc[x]], 80))
            holder[vocid] = [name_ratio, name_count, day_dif, location, rank, numships]

    return holder

def serialize_example(values):
    """
    Creates a tf.Example message ready to be written to a file.
    """
    def _float_feature(value):
        """Returns an float_list from a int/float."""
        return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    
    # Create a dictionary mapping the feature name to the tf.Example-compatible
    # data type.
    feature = {}
    for x in enumerate(values):
        feature[str(x[0] + 1)] = _float_feature(values[x[0]])

    # Create a Features message using tf.train.Example.

    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def ranking(matches, directory):
    tags=["serve"]
    signature_def_key = "predict"
    saved_model_dir = directory
    holder = []
    with Session() as sess:
        loader.load(sess, tags, saved_model_dir)
        serialized_examples = []
        vocop_ids = [x for x in matches]
        for x in matches:
            names = matches[x][0]
            days = matches[x][2]
            locations = matches[x][3]
            ranks = matches[x][4]
            ships = matches[x][5]
            name_count = matches[x][1]
            serialized_example = serialize_example([names, name_count, days, locations, 
                                                   ranks, ships] )
            serialized_examples.append(serialized_example)
            inputs_feed_dict = {'input_example_tensor:0': serialized_examples}
            outputs = sess.run('groupwise_dnn_v2/accumulate_scores/div_no_nan:0', feed_dict=inputs_feed_dict)
            output = [(outputs[x], vocop_ids[x]) for x in range(len(outputs))]
            holder = output
    return holder